In [4]:
'''
# MODEL for target pressure given temp and others, working on real time updat efor new data for some reason its called feautures

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib  # Import joblib for saving and loading models

global LOC, data_path
LOC = '/home/bkelley/capstone/data_collection/weather'
data_path = '/home/bkelley/capstone/data_collection/weather/data/hourly_weather_with_temp_avg.csv'

if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    
    # Load data
    data = pd.read_csv(data_path)
    
    # Extract feature and target arrays
    x, y = data.drop('temperature_2m', axis=1), data[['surface_pressure_Pa']]
    cats = x.select_dtypes(exclude=np.number).columns.tolist()
    
    for col in cats:
        x[col] = x[col].astype('category')

    # Split the data (size=0.25)
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

    # Create regression matrices
    dtrain_reg = xgb.DMatrix(x_train, y_train, enable_categorical=True)
    dtest_reg = xgb.DMatrix(x_test, y_test, enable_categorical=True)

    # Define hyperparameters
    params = {"objective": "reg:squarederror", "tree_method": "hist"}
    n = 200
    evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
    
    # Train the model with early stopping
    model = xgb.train(
        params=params,
        dtrain=dtrain_reg,
        num_boost_round=n,
        evals=evals,
        verbose_eval=5,
        early_stopping_rounds=50
    )
    
    print('Model training done.')

    # Save the trained model
    model_filename = 'xgb_model_pres_temp.json'  # Specify your desired filename
    model.save_model(model_filename)
    print(f'Model saved to {model_filename}.')

    # Make predictions
    preds = model.predict(dtest_reg)
    rmse = mean_squared_error(y_test, preds, squared=False)
    print(f"RMSE of the base model: {rmse:.3f}")
    print('Starting Cross - Validation....')
    # 5-fold cross-validation
    results = xgb.cv(
        params, dtrain_reg,
        num_boost_round=n,
        nfold=5,
        early_stopping_rounds=35
    )
    best_rmse = results['test-rmse-mean'].min()
    print(f'Best RMSE from CV: {best_rmse:.3f}')

# To load the model and make predictions later
loaded_model = xgb.Booster()
loaded_model.load_model(model_filename)

# Prepare new data for prediction, making sure it matches the training data format
new_data = pd.DataFrame({
    'feature_1': [value1],
    'feature_2': [value2],
    # Add all features used in training
})
new_data = random_sample
# Create DMatrix for the new data
new_dmatrix = xgb.DMatrix(new_data, enable_categorical=True)

# Make predictions with the loaded model
predictions = loaded_model.predict(new_dmatrix)
print("Predicted Pressure for new data:", predictions)
'''
0

0

In [2]:
'''
# TEST set
import pandas as pd
import xgboost as xgb

# Set random seed for reproducibility
random_seed = 42

# Required columns including 'date' but it will not be used for the model
required_columns_with_date = ['date', 'relative_humidity_2m', 'precipitation', 'rain', 'weather_code', 
                              'surface_pressure', 'cloud_cover', 'wind_speed_10m', 
                              'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 
                              'temperature_2m_K', 'surface_pressure_Pa', 'density', 'speed_of_sound']

# Grab 100 random rows with the random seed
data_path = '/home/bkelley/capstone/data_collection/weather/data/hourly_weather_with_temp_avg.csv'

data = pd.read_csv(data_path)

filtered_data = data[required_columns_with_date]

random_sample = filtered_data.sample(n=100, random_state=random_seed)

# Add placeholder 'date' column
random_sample_with_date = random_sample.copy()
random_sample_with_date['date'] = 0  # Placeholder value for 'date'

# Now you can proceed with creating the DMatrix for the random sample with 'date'
new_dmatrix = xgb.DMatrix(random_sample_with_date[required_columns_with_date], enable_categorical=True)

# Make predictions with the loaded model
predictions = loaded_model.predict(new_dmatrix)

# Grab the actual surface pressure from the dataset
actual_pressures = random_sample_with_date['surface_pressure_Pa'].values

# Grab the temperature values from the dataset
temperature_values = random_sample_with_date['temperature_2m_K'].values

# Create a DataFrame with predicted, actual, and temperature columns
results_df = pd.DataFrame({
    'Predicted Pressure': predictions,
    'Actual Pressure': actual_pressures,
    'Temperature (K)': temperature_values
})

# Convert pressures from Pa to hPa (hectopascals or millibars)
results_df['Predicted Pressure'] = results_df['Predicted Pressure'] / 100
results_df['Actual Pressure'] = results_df['Actual Pressure'] / 100

# Display the DataFrame
print(results_df)
'''
print(0)

0


In [ ]:
# FINAL DEV 
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib  # Import joblib for saving and loading models

# Global variables for paths
global LOC, data_path
LOC = '/home/bkelley/capstone/data_collection/weather'
data_path = f'{LOC}/data/hourly_weather_with_temp_avg.csv'

# Set random seed for reproducibility
random_seed = 42

if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    
    # Load data
    data = pd.read_csv(data_path)
    
    # Extract feature and target arrays
    x, y = data.drop('temperature_2m', axis=1), data[['surface_pressure_Pa']]
    cats = x.select_dtypes(exclude=np.number).columns.tolist()
    
    for col in cats:
        x[col] = x[col].astype('category')

    # Split the data (size=0.25)
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

    # Create regression matrices
    dtrain_reg = xgb.DMatrix(x_train, y_train, enable_categorical=True)
    dtest_reg = xgb.DMatrix(x_test, y_test, enable_categorical=True)

    # Define hyperparameters
    params = {"objective": "reg:squarederror", "tree_method": "hist"}
    n = 200
    evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
    
    # Train the model with early stopping
    model = xgb.train(
        params=params,
        dtrain=dtrain_reg,
        num_boost_round=n,
        evals=evals,
        verbose_eval=5,
        early_stopping_rounds=50
    )
    
    print('Model training done.')

    # Save the trained model
    model_filename = f'{LOC}/xgb_model_pres_temp.json'
    model.save_model(model_filename)
    print(f'Model saved to {model_filename}.')

    # Make predictions
    preds = model.predict(dtest_reg)
    rmse = mean_squared_error(y_test, preds, squared=False)
    print(f"RMSE of the base model: {rmse:.3f}")
    print('Starting Cross - Validation....')

    # 5-fold cross-validation
    results = xgb.cv(
        params, dtrain_reg,
        num_boost_round=n,
        nfold=5,
        early_stopping_rounds=35
    )
    best_rmse = results['test-rmse-mean'].min()
    print(f'Best RMSE from CV: {best_rmse:.3f}')

# To load the model and make predictions later
loaded_model = xgb.Booster()
loaded_model.load_model(model_filename)

# Required columns including 'date' but it will not be used for the model
required_columns_with_date = ['date', 'relative_humidity_2m', 'precipitation', 'rain', 'weather_code', 
                              'surface_pressure', 'cloud_cover', 'wind_speed_10m', 
                              'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 
                              'temperature_2m_K', 'surface_pressure_Pa', 'density', 'speed_of_sound']

# Load the data again to sample new rows
data = pd.read_csv(data_path)

# Filter the data for required columns
filtered_data = data[required_columns_with_date]

# Grab 100 random rows with the random seed
random_sample = filtered_data.sample(n=100, random_state=random_seed)

# Add placeholder 'date' column
random_sample_with_date = random_sample.copy()
random_sample_with_date['date'] = 0  # Placeholder value for 'date'

# Create DMatrix for the random sample with 'date'
new_dmatrix = xgb.DMatrix(random_sample_with_date[required_columns_with_date], enable_categorical=True)

# Make predictions with the loaded model
predictions = loaded_model.predict(new_dmatrix)

# Grab the actual surface pressure from the dataset
actual_pressures = random_sample_with_date['surface_pressure_Pa'].values

# Grab the temperature values from the dataset
temperature_values = random_sample_with_date['temperature_2m_K'].values

# Create a DataFrame with predicted, actual, and temperature columns
results_df = pd.DataFrame({
    'Predicted Pressure': predictions,
    'Actual Pressure': actual_pressures,
    'Temperature (K)': temperature_values
})

# Convert pressures from Pa to hPa (hectopascals or millibars)
results_df['Predicted Pressure'] = results_df['Predicted Pressure'] / 100
results_df['Actual Pressure'] = results_df['Actual Pressure'] / 100

# Display the DataFrame
print(results_df)


[0]	train-rmse:494.29147	validation-rmse:494.44767
[5]	train-rmse:85.12554	validation-rmse:88.19774
[10]	train-rmse:20.19447	validation-rmse:31.47241
[15]	train-rmse:12.26312	validation-rmse:27.75708
[20]	train-rmse:10.78677	validation-rmse:27.65372
[25]	train-rmse:9.92034	validation-rmse:27.59705
[30]	train-rmse:9.36951	validation-rmse:27.60585
[35]	train-rmse:8.85537	validation-rmse:27.60292
[40]	train-rmse:8.46825	validation-rmse:27.61352
[45]	train-rmse:8.08535	validation-rmse:27.61024
[50]	train-rmse:7.80327	validation-rmse:27.62141
[55]	train-rmse:7.52581	validation-rmse:27.62151
[60]	train-rmse:7.27578	validation-rmse:27.62667
[65]	train-rmse:7.08114	validation-rmse:27.63788
[70]	train-rmse:6.90646	validation-rmse:27.64899
[73]	train-rmse:6.80911	validation-rmse:27.65563
Model training done.
Model saved to /home/bkelley/capstone/data_collection/weather/xgb_model_pres_temp.json.
RMSE of the base model: 27.656
Starting Cross - Validation....


In [3]:
random_seed = 1235
# Required columns including 'date' but it will not be used for the model
required_columns_with_date = ['date', 'relative_humidity_2m', 'precipitation', 'rain', 'weather_code', 
                              'surface_pressure', 'cloud_cover', 'wind_speed_10m', 
                              'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 
                              'temperature_2m_K', 'surface_pressure_Pa', 'density', 'speed_of_sound']

# Load the data again to sample new rows
data = pd.read_csv(data_path)

# Filter the data for required columns
filtered_data = data[required_columns_with_date]

# Grab 100 random rows with the random seed
random_sample = filtered_data.sample(n=100, random_state=random_seed)

# Add placeholder 'date' column
random_sample_with_date = random_sample.copy()
random_sample_with_date['date'] = 0  # Placeholder value for 'date'

# Create DMatrix for the random sample with 'date'
new_dmatrix = xgb.DMatrix(random_sample_with_date[required_columns_with_date], enable_categorical=True)

# Make predictions with the loaded model
predictions = loaded_model.predict(new_dmatrix)

# Grab the actual surface pressure from the dataset
actual_pressures = random_sample_with_date['surface_pressure_Pa'].values

# Grab the temperature values from the dataset
temperature_values = random_sample_with_date['temperature_2m_K'].values

# Create a DataFrame with predicted, actual, and temperature columns
results_df = pd.DataFrame({
    'Predicted Pressure': predictions,
    'Actual Pressure': actual_pressures,
    'Temperature (K)': temperature_values
})

# Convert pressures from Pa to hPa (hectopascals or millibars)
results_df['Predicted Pressure'] = results_df['Predicted Pressure'] / 100
results_df['Actual Pressure'] = results_df['Actual Pressure'] / 100

# Display the DataFrame
print(results_df)


NameError: name 'pd' is not defined